<a href="https://colab.research.google.com/github/MOHOAzure/AI_Paint_Online/blob/main/Colab/WebUI_My_Models_Collections.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Thanks for your contributions

藏經閣
https://drive.google.com/drive/folders/1vGc16Bb8CDW1piUj_5thzbsCmDFamYyt

In [1]:
# !pip list

In [2]:
#@title Setup Tool to use models on My Drive

# (optional: check GPU)
# !nvidia-smi

from pathlib import Path

# param from user
version = 'latest' #@param ["latest", "685f9631b56ff8bd43bce24ff5ce0f9a0e9af490"] {allow-input: true}

# model selection
model_and_misc_dir = '/content/drive/Shareddrives/AI_Paint/stable-diffusion' #@param ["/content/drive/Shareddrives/AI_Paint/stable-diffusion", "/content/drive/MyDrive/stable-diffusion"] {allow-input: true}

## the model dir where the tool can find models
model_dir = "models" #@param {type:"string"}

## specific model under the model dir, including extension
model_filename = "meinamix_meinaV51.safetensors" #@param {type:"string"}

use_specific_model = False #@param {type:"boolean"}

use_cuda = False # @param {type:"boolean"}

# define variables such as working directory
root_dir = Path("/content")
tool_dir = root_dir/"stable-diffusion-webui"
model_and_misc_dir = Path(model_and_misc_dir)
model_dir = model_and_misc_dir/model_dir

# load Google Drive for my models
from google.colab import drive
drive.mount('/content/drive')

def use_cuda():
  # use cuda instead of cpu for large model (e.g., 7G)
  !sed -i 's/weight_load_location = None if cmd_opts.lowram else "cpu"/weight_load_location = None if cmd_opts.lowram else "cuda"/g' {tool_dir}/modules/shared.py

def install_dep():
  # install xformers & its dep.
  %cd {root_dir}
  !pip install xformers==0.0.16

  # !pip install xformers==0.0.17rc482 # require: torch 2.0.0. fast calc
  # !pip install torchvision==0.15.1
  # !pip install torchtext==0.15.1
  # !pip install ㄝ==2.0.1
  ## !pip install open-clip-torch==2.16.0

  # !python -m xformers.info

  # old versions
  !pip install tensorflow==2.9.2
  !pip install fastapi==0.90.0

def install_extentions():
  %cd {tool_dir}/extensions
  
  # Commonly used extention
  !git clone https://github.com/DominikDoom/a1111-sd-webui-tagcomplete.git
  # !git clone https://github.com/hnmr293/sd-webui-llul.git 
  # !git clone https://github.com/toriato/stable-diffusion-webui-wd14-tagger
  # !git clone https://github.com/ThereforeGames/unprompted.git
  # !git clone https://github.com/hnmr293/sd-webui-cutoff.git
  # !git clone https://github.com/7eu7d7/DreamArtist-sd-webui-extension.git
  # !git clone https://github.com/bbc-mc/sdweb-merge-board
  # !git clone https://github.com/tkalayci71/embedding-inspector
  # !git clone https://github.com/bbc-mc/sdweb-merge-block-weighted-gui
  # !git clone https://github.com/Malisius/booru2prompt.git
  # !git clone https://github.com/adieyal/sd-dynamic-prompts
  # !git clone https://github.com/kousw/stable-diffusion-webui-daam
  # !git clone https://github.com/Akegarasu/sd-webui-model-converter.git

%cd $root_dir

# install tool: stable-diffusion-webui
repo = "https://github.com/AUTOMATIC1111/stable-diffusion-webui.git"
!git clone $repo

# time machine: commit or branch
if version:
  !git checkout $version

install_dep()
install_extentions()

# load tool config
!cp {model_and_misc_dir}/"[template] ui-config.json" {tool_dir}/ui-config.json
!cp {model_and_misc_dir}/"[template] config.json" {tool_dir}/config.json

if use_specific_model:
  model = model_dir / model_filename

# additional VAE
# !cp /content/drive/MyDrive/stable-diffusion/models/VAE/orangemix.vae.pt {tool_dir}/models/VAE/
# embedding
# !cp /content/drive/MyDrive/stable-diffusion/models/embeddings/* {tool_dir}/embeddings/

if use_cuda:
  use_cuda()

Mounted at /content/drive
/content
Cloning into 'stable-diffusion-webui'...
remote: Enumerating objects: 17850, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 17850 (delta 0), reused 0 (delta 0), pack-reused 17846
Receiving objects: 100% (17850/17850), 28.23 MiB | 16.50 MiB/s, done.
Resolving deltas: 100% (12499/12499), done.
fatal: not a git repository (or any of the parent directories): .git
/content
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 MB 13.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.8/511.8 MB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 KB 7.0 MB/s eta 0:00:00
     ━━━━━━

In [3]:
#@title Install extension: Latent Couple - 2 shot
use_latent_couple = False # @param {type:"boolean"}
if use_latent_couple:
  %cd {tool_dir}/extensions
  !git clone https://github.com/opparco/stable-diffusion-webui-two-shot.git
  # !git clone https://github.com/miZyind/sd-webui-latent-couple.git
  # %cd {tool_dir}
  # !git apply --ignore-whitespace extensions/sd-webui-latent-couple/cfg_denoised_callback-ea9bd9fc.patch
  # !git apply --ignore-whitespace extensions/sd-webui-latent-couple/0001-Adding-after_ui_callback-for-scripts.patch

In [4]:
#@title Install extension: supermerge
use_supermerge = False # @param {type:"boolean"}
if use_supermerge:
  %cd {tool_dir}/extensions  
  !git clone https://github.com/hako-mikan/sd-webui-supermerger.git

In [5]:
#@title Install extension: controlnet

use_controlnet = False # @param {type:"boolean"}

if use_controlnet:
  # @markdown ---
  # @markdown - Info: ControlNet
  # @markdown  - https://github.com/lllyasviel/ControlNet
  # @markdown  - https://github.com/Mikubill/sd-webui-controlnet
  # @markdown  - https://huggingface.co/webui/ControlNet-modules-safetensors/

  %cd {tool_dir}/extensions
  !git clone https://github.com/Mikubill/sd-webui-controlnet.git
  # !git clone https://github.com/fkunn1326/openpose-editor.git
  !git clone https://github.com/nonnonstop/sd-webui-3d-open-pose-editor.git

  control_seg = False # @param {type:"boolean"}
  control_openpose = True # @param {type:"boolean"}
  control_canny = False # @param {type:"boolean"}
  control_depth = False # @param {type:"boolean"}
  control_hed = False # @param {type:"boolean"}
  control_mlsd = False # @param {type:"boolean"}
  control_normal = False # @param {type:"boolean"}
  control_scribble = False # @param {type:"boolean"}
  t2iadapter_keypose = False # @param {type:"boolean"}
  t2iadapter_openpose = False # @param {type:"boolean"}
  t2iadapter_seg = False # @param {type:"boolean"}
  t2iadapter_sketch = False # @param {type:"boolean"}

  control_net_models = []
  if t2iadapter_keypose == True:
      control_net_models.append("t2iadapter_keypose")
  if t2iadapter_seg == True:
      control_net_models.append("t2iadapter_seg")
  if t2iadapter_sketch == True:
      control_net_models.append("t2iadapter_sketch")
  if t2iadapter_openpose == True:
      control_net_models.append("t2iadapter_openpose")
  if control_canny == True:
      control_net_models.append("control_canny")
  if control_depth == True:
      control_net_models.append("control_depth")
  if control_hed == True:
      control_net_models.append("control_hed")
  if control_mlsd == True:
      control_net_models.append("control_mlsd")
  if control_normal == True:
      control_net_models.append("control_normal")
  if control_openpose == True:
      control_net_models.append("control_openpose")
  if control_scribble == True:
      control_net_models.append("control_scribble")
  if control_seg == True:
      control_net_models.append("control_seg")

  if Path('sd-webui-controlnet').is_dir():
      %cd sd-webui-controlnet/models
      for control_net_model in control_net_models:
        ! wget -nc --content-disposition https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/{control_net_model}-fp16.safetensors

# Launch
Provide a directory to models (sub dir is optional) and then run this script. You will get something like xxx.gradio.app, click it to go to the deployed Web UI.

---

This script use some parameters to run Web UI. The command line parameters are available at [stable-diffusion-webui Wiki](https://github.com/AUTOMATIC1111/stable-diffusion-webui/wiki/Command-Line-Arguments-and-Settings)

The following parameters are the most important ones for use via colab & gradio:
  - `--share` - create an online gradio.app link
  - `--gradio-debug` - print output to console
  - `--gradio-auth YOUR_ACCOUNT:YOUR_PWD` - create a login account

*   In order to prevent others from crawling your online deployment for use, it is strongly recommended to modify the username and password, and `replace account & password with others`.

---
My Quality Prompt
> 1girl, (close to viewer:1.4), Perfect details,extremely fine and beautiful, Amazing,beautiful background,finely detail,Depth of field,extremely detailed 8k wallpaper,(magnificent), ((best quality)), ((masterpiece)), ((ultra-detailed)), ((an extremely delicate and beautiful),((extremely detailed)), optical, reflective, epic,Hyperdetail, global illumination, epic scenes, beautiful and delicate

> beautiful complex, oil on canvas, (masterpiece:1.3),(best quality,ultra-detailed,illustration:1.2), (extremely detailed cg,8k,high resolution), flawless, clean, professional artwork, famous artwork, cinematic lighting, cinematic bloom, studio photo, sharp focus


My Negative Prompts:
> (frown:1.5), (worst quality, low quality:1.4), (lip, nose, tooth, rouge, lipstick, eyeshadow:1.0), (dusty sunbeams:1.0), (abs, muscular, rib:1.0), (depth of field, bokeh, blurry:1.4), (greyscale, monochrome:1.0), (text, title, logo, signature:1.2)

> (lowres, blurry, worst quality, low quality:1.21), normal quality, multiple breasts, (mutated hands and fingers:1.5), (long body:1.3), (mutation, poorly drawn:1.2), black-white, bad anatomy, disfigured, deformed, mutation, mutilated, ugly, poorly drawn face, cloned face, (unclear blurry eyes:1.331), bad eyes, fused eyes, poorly drawn eyes, extra eyes, long neck, bad hands, poorly drawn hands, multiple limbs, extra limbs, malformed limbs, missing arms, missing fingers, interlocked fingers, extra fingers, fused fingers, too many fingers, long nails, missing legs, extra legs, broken legs, bad feet, extra digit, fewer digits, glitchy, (jpeg artifacts, ((signature, watermark, username)), text, error, multiple_views, reference_sheet)

Common Negative Prompts:
> lowres, blurry, worst quality, low quality, normal quality,bad anatomy, disfigured, deformed, mutation, mutilated, ugly, totem pole,poorly drawn face,  cloned face, several faces, long neck, mutated hands, bad hands, poorly drawn hands,extra limbs, malformed limbs, missing arms, missing fingers, extra fingers, fused fingers, too many fingers,missing legs, extra legs, malformed legs, extra digit, fewer digits, glitchy, cropped, jpeg artifacts, signature, watermark, username, text, error

> plain background, poorly drawn face, poorly drawn hands, watermark, censored, (mutated hands and fingers), ugly

In [ ]:
#@title Launch
%cd {tool_dir}

# security
account = "MY_ACCOUNT" #@param {type:"string"}
pwd = "MY_PWD" #@param {type:"string"}

load_model_cmd = f"--ckpt-dir {model_dir}"

if use_specific_model:
  load_model_cmd = f"--ckpt {model}"
else:
  # update the first model to load
  !sed -i "s#THE_FIRST_MODEL.ext#{model_filename}#g" {tool_dir}/config.json

# use param to run tool
# ref: https://github.com/AUTOMATIC1111/stable-diffusion-webui/wiki/Command-Line-Arguments-and-Settings
# --no-progressbar-hiding (may slow down)
# Speed to gen pic: --opt-split-attention OR --xformers
# --no-hashing
arg = f"--xformers --no-half-vae --share --gradio-debug --gradio-auth {account}:{pwd} --disable-safe-unpickle --enable-insecure-extension-access {load_model_cmd}"

!COMMANDLINE_ARGS="{arg}" REQS_FILE="requirements.txt" python launch.py

Loading VAE weights specified in settings: /content/drive/Shareddrives/AI_Paint/stable-diffusion/models/marbal_models/pastel-waifu-diffusion.vae.pt
Applying xformers cross attention optimization.
VAE weights loaded.
